In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import manifold
from glob import glob
import pandas as pd
from sklearn.model_selection import train_test_split

# 2D t-SNE

In [ ]:
all_data_set_name = [ 'ASSIST2009NSB', 'ASSIST2012', 'EdNet']
data_all = []
select_n = 1000
for one_data_set in all_data_set_name:
    DATA_NAME = one_data_set
    processed_data = pd.read_pickle(f'raw_data/{DATA_NAME}.pkl')
    all_stu = processed_data['stu_id'].value_counts().index
    X_test, use_stu = train_test_split(all_stu, random_state=2022, test_size=select_n, shuffle=True)
    one_data = processed_data[processed_data['stu_id'].isin(use_stu)]
    for one_stu in use_stu:
        one_row_data = [DATA_NAME,one_stu]
        one_stu_data = one_data[one_data['stu_id']==one_stu]
        one_row_data.extend([len(one_stu_data['stu_id']),np.mean(one_stu_data['is_right']),len(set(list(one_stu_data['q_id']))),
                        len(set(list(one_stu_data['kp_id']))),np.mean(one_stu_data['use_time'])])
        data_all.append(one_row_data)
all_data_set = pd.DataFrame(data_all,columns=['dataset','stu_id','records','acc','questions','kps','use_time'])
use_col = [ 'records','acc','questions','kps','use_time']
brats_array = all_data_set[use_col]
brats_label_array = np.array(all_data_set['dataset'])
tsne = manifold.TSNE(n_components=2,learning_rate='auto', init='pca', random_state=42).fit_transform(brats_array)
x_min, x_max = tsne.min(0), tsne.max(0)
tsne_norm = (tsne - x_min) / (x_max - x_min)
tsne_norm = tsne_norm*100
data = []
for one in tsne_norm:
    if one[0]<2.1 and one[1]<18:
        data.append(one)
import pyecharts.options as opts
from pyecharts.charts import Scatter
scar = (
    Scatter(init_opts=opts.InitOpts(width="600px", height="600px"))
    .set_series_opts()
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(
            name = 'comp-1',
            type_="value", splitline_opts=opts.SplitLineOpts(is_show=True)
        ),
        yaxis_opts=opts.AxisOpts(
            name = 'comp-2',
            type_="value",
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        tooltip_opts=opts.TooltipOpts(is_show=False),
    )
)
for i,one in enumerate(all_data_set_name):
    y_data = [float(d[1]) for d in data][i*select_n:(i+1)*select_n]
    x_data = [float(d[0]) for d in data][i*select_n:(i+1)*select_n]
    scar.add_xaxis(xaxis_data=x_data).add_yaxis(
        series_name=one,
        y_axis=y_data,
        symbol_size=8,
        label_opts=opts.LabelOpts(is_show=False),)
scar.render('2D_t-SNE.html')
scar.render_notebook()

# 3D t-SNE

In [ ]:
all_data_set_name = [ 'ASSIST2009NSB', 'ASSIST2012','EdNet']
data_all = []
select_n = 1000
for one_data_set in all_data_set_name:
    DATA_NAME = one_data_set
    processed_data = pd.read_pickle(f'raw_data/{DATA_NAME}.pkl')
    all_stu = processed_data['stu_id'].value_counts().index
    X_test, use_stu = train_test_split(all_stu, random_state=2022, test_size=select_n, shuffle=True)
    one_data = processed_data[processed_data['stu_id'].isin(use_stu)]
    for one_stu in use_stu:
        one_row_data = [DATA_NAME,one_stu]
        one_stu_data = one_data[one_data['stu_id']==one_stu]
        one_row_data.extend([len(one_stu_data['stu_id']),np.mean(one_stu_data['is_right']),len(set(list(one_stu_data['q_id']))),
                        len(set(list(one_stu_data['kp_id']))),np.mean(one_stu_data['use_time'])])
        data_all.append(one_row_data)
all_data_set = pd.DataFrame(data_all,columns=['dataset','stu_id','records','acc','questions','kps','use_time'])
use_col = [ 'records','acc','questions','kps','use_time']
brats_array = all_data_set[use_col]
brats_label_array = np.array(all_data_set['dataset'])
tsne = manifold.TSNE(n_components=3,learning_rate='auto', init='pca', random_state=42).fit_transform(brats_array)
x_min, x_max = tsne.min(0), tsne.max(0)
tsne_norm = (tsne - x_min) / (x_max - x_min)
tsne_norm = tsne_norm*100
datas = tsne_norm
data = []
for one in tsne_norm:
    if one[0]<20 and one[1]<40 and one[2]<100:
        data.append(one)
datas = data
import pyecharts.options as opts
from pyecharts.charts import Scatter3D
color_set = {'ASSIST2009NSB':'red', 'ASSIST2012':'blue', 'EdNet':'green'}

import pyecharts.options as opts
from pyecharts.charts import Scatter3D
scatt = (
    Scatter3D(
        init_opts=opts.InitOpts(width="800px", height="800px")
    )  # bg_color="black"
)
for i,one in enumerate(all_data_set_name):
    data = [[float(itme[2]),float(itme[1]),float(itme[0]),color_set[brats_label_array[index]]] for index,itme in enumerate(datas[i*select_n:(i+1)*select_n])]
    scatt.add(
        series_name=one,
        data=data,
        xaxis3d_opts=opts.Axis3DOpts(
            name='comp-2',
            type_="value",
        ),
        yaxis3d_opts=opts.Axis3DOpts(
            name='comp-1',
            type_="value",
        ),
        zaxis3d_opts=opts.Axis3DOpts(
            name='comp-3',
            type_="value",
        ),
        grid3d_opts=opts.Grid3DOpts(width=100, height=100, depth=100),  
    )
scatt.render('3D_t-SNE.html')
scatt.render_notebook()